In [5]:
#takes a file and splits it by record name
from Bio import SeqIO
for record in SeqIO.parse("~/Downloads/sar202.viral.contigs.10k.cat12.fa", "fasta"):
    print(record.name)
    SeqIO.write(record, record.name+".fasta", "fasta")

FileNotFoundError: [Errno 2] No such file or directory: 'sar202.viral.contigs.10k.cat12.fa'

In [ ]:
#reformat the fasta file of viral contigs

source activate anvio5
for i in $(cat contig-names.txt)
do 
anvi-script-reformat-fasta $i.fasta -o reformatted/$i.fasta --simplify-names --prefix $i
done

cat *.fasta > all_sar202_contigs.fasta

In [ ]:
#build a viral database
bowtie2-build viral_contigs/reformatted/all_sar202_contigs.fasta sar202-viral

#profile the viral stuff

#!/bin/bash
#PBS -V # set verbose output
#PBS -N SAR202-viral-mappings
#PBS -d /gpfs/ts0/home/bt273/BIOS-SCOPE/metag/ashley/SAR202
#PBS -q pq
#PBS -l walltime=96:00:00 # Maximum wall time for the job. 
#PBS -A Research_Project-172179 # research project to submit under.
#PBS -l procs=16 # either this to specify number of processors.
#PBS -j oe
#PBS -l feature=phi     # Phi

PROJECT_HOME=/gpfs/ts0/home/bt273/BIOS-SCOPE/metag/ashley/SAR202
OUT_DIR=$PROJECT_HOME/viral_mappings
READS=/gpfs/ts0/home/bt273/BIOS-SCOPE/metag/ashley/SAR202/reads

SAMPLE_NAME=sar202-viral

MAPPING_LOGFILE=$OUT_DIR/$SAMPLE_NAME.log

cd $PROJECT_HOME

for i in $(cat filenames.txt);
do source activate anvio5;
bowtie2 --threads 16 -x $SAMPLE_NAME -1 $READS/$i.fwd* -2 $READS/$i.rev* --no-unal -S $SAMPLE_NAME.$i.sam 2>&1 | tee -a $MAPPING_LOGFILE;

samtools view -F 4 -bS $SAMPLE_NAME.$i.sam > $SAMPLE_NAME.vs.$i.raw.bam;
samtools sort $SAMPLE_NAME.vs.$i.raw.bam -o $SAMPLE_NAME.vs.$i.sorted.bam;
samtools index $SAMPLE_NAME.vs.$i.sorted.bam;

source activate bamm;

bamm filter -b $SAMPLE_NAME.vs.$i.sorted.bam --percentage_id 0.95 2>&1 | tee -a $MAPPING_LOGFILE;
samtools sort -@ 16 \
-o $SAMPLE_NAME.vs.$i.sorted.95pc.bam \
$SAMPLE_NAME.vs.$i.sorted_filtered.bam;

samtools index $SAMPLE_NAME.vs.$i.sorted.95pc.bam;
mv $SAMPLE_NAME.vs.$i.sorted.95pc.bam* $OUT_DIR;

rm *$SAMPLE_NAME.vs.$i*;
rm *$SAMPLE_NAME.$i.sam;
done

In [ ]:
bowtie2-build sar202_contigs/reformatted/all_sar202_contigs.fasta sar202

#!/bin/bash
#PBS -V # set verbose output
#PBS -N sar202-biller-bowtie-bamm
#PBS -d /gpfs/ts0/home/bt273/BIOS-SCOPE/metag/ashley/SAR202
#PBS -q pq
#PBS -l walltime=96:00:00 # Maximum wall time for the job. 
#PBS -A Research_Project-172179 # research project to submit under.
#PBS -l procs=16 # either this to specify number of processors.
#PBS -j oe
#PBS -l feature=phi     # Phi

PROJECT_HOME=/gpfs/ts0/home/bt273/BIOS-SCOPE/metag/ashley/SAR202
OUT_DIR=$PROJECT_HOME/mappings
READS=/gpfs/ts0/home/bt273/BIOS-SCOPE/metag/ashley/SAR202/reads

SAMPLE_NAME=sar202

MAPPING_LOGFILE=$OUT_DIR/$SAMPLE_NAME.log

cd $PROJECT_HOME

for i in $(cat filenames.txt);
do source activate anvio5;
bowtie2 --threads 16 -x $SAMPLE_NAME -1 $READS/$i.fwd* -2 $READS/$i.rev* --no-unal -S $SAMPLE_NAME.$i.sam 2>&1 | tee -a $MAPPING_LOGFILE;

samtools view -F 4 -bS $SAMPLE_NAME.$i.sam > $SAMPLE_NAME.vs.$i.raw.bam;
samtools sort $SAMPLE_NAME.vs.$i.raw.bam -o $SAMPLE_NAME.vs.$i.sorted.bam;
samtools index $SAMPLE_NAME.vs.$i.sorted.bam;

source activate bamm;

bamm filter -b $SAMPLE_NAME.vs.$i.sorted.bam --percentage_id 0.95 2>&1 | tee -a $MAPPING_LOGFILE;
samtools sort -@ 16 \
-o $SAMPLE_NAME.vs.$i.sorted.95pc.bam \
$SAMPLE_NAME.vs.$i.sorted_filtered.bam;

samtools index $SAMPLE_NAME.vs.$i.sorted.95pc.bam;
mv $SAMPLE_NAME.vs.$i.sorted.95pc.bam* $OUT_DIR;

rm *$SAMPLE_NAME.vs.$i*;
rm *$SAMPLE_NAME.$i.sam;
done

In [ ]:
#!/bin/bash
#PBS -V # set verbose output
#PBS -N sar202-tpmeans
#PBS -d /gpfs/ts0/home/bt273/BIOS-SCOPE/metag/ashley/SAR202/mappings
#PBS -q pq
#PBS -l walltime=8:00:00 # Maximum wall time for the job. 
#PBS -A Research_Project-172179 # research project to submit under.
#PBS -l procs=32 # either this to specify number of processors.
#PBS -j oe
#PBS -l feature=highmem # High Memory

source activate bamm
bamm parse \
--coverages sar202-cellular-coverage.tsv \
--coverage_mode tpmean \
--bamfiles *.bam \
--threads 32 \

In [ ]:
#checkM
#https://github.com/Ecogenomics/CheckM/wiki/Genome-Quality-Commands

PATH=$PATH:/gpfs/ts0/home/bt273/BIOS-SCOPE/metag/ashley/tools/Prodigal
source activate py27

checkm lineage_wf sar202_contigs/ checkM_output/

cd SAGs/
checkm taxon_set phylum Chloroflexi chloroflexi_markers
checkm analyze chloroflexi_markers sar202_contigs checkM_results -x fasta -t 16
checkm qa --out_format 2 --file completeness_stats.txt --tab_table -t 16 chloroflexi_markers checkM_results

In [ ]:
#set up diamond
wget ftp://ftp.ncbi.nlm.nih.gov/blast/db/FASTA/nr.gz
wget ftp://ftp.ncbi.nlm.nih.gov/pub/taxonomy/accession2taxid/prot.accession2taxid.gz
wget ftp://ftp.ncbi.nlm.nih.gov/pub/taxonomy/taxdmp.zip

source activate anvio5
diamond makedb --in nr.gz --db nr --taxonmap prot.accession2taxid.gz --taxonnodes nodes.dmp

#run diamond blastx on the viral contigs to find identities
diamond blastx \
-d ../../tools/diamond/nr.dmnd \
-q ../viral_contigs/reformatted/all_sar202_contigs.fasta \
-o diamond-taxonomy.tsv \
--taxonmap ../../tools/diamond/prot.accession2taxid.gz \
--taxonnodes ../../tools/diamond/nodes.dmp \
--more-sensitive \
--outfmt 102

In [ ]:
#prokka
https://github.com/tseemann/prokka
conda install -n anvio5 -c conda-forge -c bioconda prokka

#source activate anvio5

for i in $(cat viral-contigs.txt);
do new_name=$(basename $i .fasta);
prokka $i \
--outdir $new_name \
--prefix $new_name \
--force \
--kingdom Virus \
--metagenome \
--compliant \
--evalue 10.0 \
--cpus 0 \
--genus $new_name \
--species  \
--strain  \
--locustag Gene \
--addgenes \
--rfam;
done

for i in $(cat viral-contigs.txt);
do new_name=$(basename $i .fasta);
cp $new_name/$new_name.gbk gbk_files/;
done

In [ ]:
for i in `ls `; do ln -s $i/*.fq.gz .; done #make symbolic links

for i in `ls *.fq.gz` ;
do echo $i >> reads.txt ;
zcat $i | grep -c '^+' >> reads.txt;
done

In [47]:
import pandas as pd
df = pd.read_csv("/home/minion/Desktop/holger/sar202_all_cmeans_coverage.tsv", sep = '\t')
df

,#contig,Length,sar202.vs.SRR5720222.sorted.95pc.bam,sar202.vs.SRR5720219.sorted.95pc.bam,sar202.vs.SRR5720227.sorted.95pc.bam,sar202.vs.SRR5720228.sorted.95pc.bam,sar202.vs.SRR5720220.sorted.95pc.bam,sar202.vs.SRR5720223.sorted.95pc.bam,sar202.vs.SRR5720221.sorted.95pc.bam,sar202.vs.SRR5720230.sorted.95pc.bam,...,sar202.vs.SRR5788455.sorted.95pc.bam,sar202.vs.SRR5788468.sorted.95pc.bam,sar202.vs.SRR6507279.sorted.95pc.bam,sar202.vs.SRR5788464.sorted.95pc.bam,sar202.vs.SRR6507277.sorted.95pc.bam,sar202.vs.SRR5788452.sorted.95pc.bam,sar202.vs.SRR5788456.sorted.95pc.bam,sar202.vs.SRR6507278.sorted.95pc.bam,sar202.vs.SRR6507280.sorted.95pc.bam,sar202.vs.SRR5788467.sorted.95pc.bam
0,LSU_GOM1_000000000001,17893,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,...,0.0000,0.0,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
1,LSU_GOM1_000000000002,3164,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,...,0.0000,0.0,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2,LSU_GOM1_000000000003,14152,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,...,0.0000,0.0,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
3,LSU_GOM1_000000000004,5298,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,...,0.0000,0.0,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
4,LSU_GOM1_000000000005,18323,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,...,0.0000,0.0,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
5,LSU_GOM1_000000000006,8039,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,...,0.0000,0.0,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
6,LSU_GOM1_000000000007,7574,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,...,0.0000,0.0,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
7,LSU_GOM1_000000000008,5392,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,...,0.0000,0.0,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
8,LSU_GOM1_000000000009,19342,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,...,0.0000,0.0,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
9,LSU_GOM1_000000000010,4755,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,...,0.0000,0.0,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000


In [48]:
df.rename(columns={'#contig':'#contig',
'Length':'Length',
'sar202.vs.SRR5720222.sorted.95pc.bam':'SRR5720222',
'sar202.vs.SRR5720219.sorted.95pc.bam':'SRR5720219',
'sar202.vs.SRR5720227.sorted.95pc.bam':'SRR5720227',
'sar202.vs.SRR5720228.sorted.95pc.bam':'SRR5720228',
'sar202.vs.SRR5720220.sorted.95pc.bam':'SRR5720220',
'sar202.vs.SRR5720223.sorted.95pc.bam':'SRR5720223',
'sar202.vs.SRR5720221.sorted.95pc.bam':'SRR5720221',
'sar202.vs.SRR5720230.sorted.95pc.bam':'SRR5720230',
'sar202.vs.SRR5720225.sorted.95pc.bam':'SRR5720225',
'sar202.vs.SRR5720235.sorted.95pc.bam':'SRR5720235',
'sar202.vs.SRR5720233.sorted.95pc.bam':'SRR5720233',
'sar202.vs.SRR5720224.sorted.95pc.bam':'SRR5720224',
'sar202.vs.SRR5720226.sorted.95pc.bam':'SRR5720226',
'sar202.vs.SRR5720237.sorted.95pc.bam':'SRR5720237',
'sar202.vs.SRR5720231.sorted.95pc.bam':'SRR5720231',
'sar202.vs.SRR5720238.sorted.95pc.bam':'SRR5720238',
'sar202.vs.SRR5720241.sorted.95pc.bam':'SRR5720241',
'sar202.vs.SRR5720232.sorted.95pc.bam':'SRR5720232',
'sar202.vs.SRR5720229.sorted.95pc.bam':'SRR5720229',
'sar202.vs.SRR5720234.sorted.95pc.bam':'SRR5720234',
'sar202.vs.SRR5720246.sorted.95pc.bam':'SRR5720246',
'sar202.vs.SRR5720236.sorted.95pc.bam':'SRR5720236',
'sar202.vs.SRR5720240.sorted.95pc.bam':'SRR5720240',
'sar202.vs.SRR5720244.sorted.95pc.bam':'SRR5720244',
'sar202.vs.SRR5720247.sorted.95pc.bam':'SRR5720247',
'sar202.vs.SRR5720251.sorted.95pc.bam':'SRR5720251',
'sar202.vs.SRR5720245.sorted.95pc.bam':'SRR5720245',
'sar202.vs.SRR5720242.sorted.95pc.bam':'SRR5720242',
'sar202.vs.SRR5720249.sorted.95pc.bam':'SRR5720249',
'sar202.vs.SRR5720252.sorted.95pc.bam':'SRR5720252',
'sar202.vs.SRR5720239.sorted.95pc.bam':'SRR5720239',
'sar202.vs.SRR5720257.sorted.95pc.bam':'SRR5720257',
'sar202.vs.SRR5720253.sorted.95pc.bam':'SRR5720253',
'sar202.vs.SRR5720243.sorted.95pc.bam':'SRR5720243',
'sar202.vs.SRR5720250.sorted.95pc.bam':'SRR5720250',
'sar202.vs.SRR5720260.sorted.95pc.bam':'SRR5720260',
'sar202.vs.SRR5720262.sorted.95pc.bam':'SRR5720262',
'sar202.vs.SRR5720256.sorted.95pc.bam':'SRR5720256',
'sar202.vs.SRR5720248.sorted.95pc.bam':'SRR5720248',
'sar202.vs.SRR5720255.sorted.95pc.bam':'SRR5720255',
'sar202.vs.SRR5720266.sorted.95pc.bam':'SRR5720266',
'sar202.vs.SRR5720263.sorted.95pc.bam':'SRR5720263',
'sar202.vs.SRR5720258.sorted.95pc.bam':'SRR5720258',
'sar202.vs.SRR5720268.sorted.95pc.bam':'SRR5720268',
'sar202.vs.SRR5720269.sorted.95pc.bam':'SRR5720269',
'sar202.vs.SRR5720271.sorted.95pc.bam':'SRR5720271',
'sar202.vs.SRR5720272.sorted.95pc.bam':'SRR5720272',
'sar202.vs.SRR5720264.sorted.95pc.bam':'SRR5720264',
'sar202.vs.SRR5720273.sorted.95pc.bam':'SRR5720273',
'sar202.vs.SRR5720265.sorted.95pc.bam':'SRR5720265',
'sar202.vs.SRR5720261.sorted.95pc.bam':'SRR5720261',
'sar202.vs.SRR5720276.sorted.95pc.bam':'SRR5720276',
'sar202.vs.SRR5720278.sorted.95pc.bam':'SRR5720278',
'sar202.vs.SRR5720277.sorted.95pc.bam':'SRR5720277',
'sar202.vs.SRR5720270.sorted.95pc.bam':'SRR5720270',
'sar202.vs.SRR5720275.sorted.95pc.bam':'SRR5720275',
'sar202.vs.SRR5720267.sorted.95pc.bam':'SRR5720267',
'sar202.vs.SRR5720283.sorted.95pc.bam':'SRR5720283',
'sar202.vs.SRR5720280.sorted.95pc.bam':'SRR5720280',
'sar202.vs.SRR5720259.sorted.95pc.bam':'SRR5720259',
'sar202.vs.SRR5720286.sorted.95pc.bam':'SRR5720286',
'sar202.vs.SRR5720285.sorted.95pc.bam':'SRR5720285',
'sar202.vs.SRR5720274.sorted.95pc.bam':'SRR5720274',
'sar202.vs.SRR5720254.sorted.95pc.bam':'SRR5720254',
'sar202.vs.SRR5720288.sorted.95pc.bam':'SRR5720288',
'sar202.vs.SRR5720291.sorted.95pc.bam':'SRR5720291',
'sar202.vs.SRR5720287.sorted.95pc.bam':'SRR5720287',
'sar202.vs.SRR5720290.sorted.95pc.bam':'SRR5720290',
'sar202.vs.SRR5720293.sorted.95pc.bam':'SRR5720293',
'sar202.vs.SRR5720282.sorted.95pc.bam':'SRR5720282',
'sar202.vs.SRR5720279.sorted.95pc.bam':'SRR5720279',
'sar202.vs.SRR5720284.sorted.95pc.bam':'SRR5720284',
'sar202.vs.SRR5720296.sorted.95pc.bam':'SRR5720296',
'sar202.vs.SRR5720298.sorted.95pc.bam':'SRR5720298',
'sar202.vs.SRR5720299.sorted.95pc.bam':'SRR5720299',
'sar202.vs.SRR5720289.sorted.95pc.bam':'SRR5720289',
'sar202.vs.SRR5720302.sorted.95pc.bam':'SRR5720302',
'sar202.vs.SRR5720295.sorted.95pc.bam':'SRR5720295',
'sar202.vs.SRR5720294.sorted.95pc.bam':'SRR5720294',
'sar202.vs.SRR5720300.sorted.95pc.bam':'SRR5720300',
'sar202.vs.SRR5720281.sorted.95pc.bam':'SRR5720281',
'sar202.vs.SRR5720305.sorted.95pc.bam':'SRR5720305',
'sar202.vs.SRR5720307.sorted.95pc.bam':'SRR5720307',
'sar202.vs.SRR5720308.sorted.95pc.bam':'SRR5720308',
'sar202.vs.SRR5720306.sorted.95pc.bam':'SRR5720306',
'sar202.vs.SRR5720303.sorted.95pc.bam':'SRR5720303',
'sar202.vs.SRR5720301.sorted.95pc.bam':'SRR5720301',
'sar202.vs.SRR5720297.sorted.95pc.bam':'SRR5720297',
'sar202.vs.SRR5720292.sorted.95pc.bam':'SRR5720292',
'sar202.vs.SRR5720314.sorted.95pc.bam':'SRR5720314',
'sar202.vs.SRR5720312.sorted.95pc.bam':'SRR5720312',
'sar202.vs.SRR5720317.sorted.95pc.bam':'SRR5720317',
'sar202.vs.SRR5720310.sorted.95pc.bam':'SRR5720310',
'sar202.vs.SRR5720309.sorted.95pc.bam':'SRR5720309',
'sar202.vs.SRR5720313.sorted.95pc.bam':'SRR5720313',
'sar202.vs.SRR5720321.sorted.95pc.bam':'SRR5720321',
'sar202.vs.SRR5720315.sorted.95pc.bam':'SRR5720315',
'sar202.vs.SRR5720304.sorted.95pc.bam':'SRR5720304',
'sar202.vs.SRR5720320.sorted.95pc.bam':'SRR5720320',
'sar202.vs.SRR5720311.sorted.95pc.bam':'SRR5720311',
'sar202.vs.SRR5720325.sorted.95pc.bam':'SRR5720325',
'sar202.vs.SRR5720327.sorted.95pc.bam':'SRR5720327',
'sar202.vs.SRR5720316.sorted.95pc.bam':'SRR5720316',
'sar202.vs.SRR5720319.sorted.95pc.bam':'SRR5720319',
'sar202.vs.SRR5720330.sorted.95pc.bam':'SRR5720330',
'sar202.vs.SRR5720329.sorted.95pc.bam':'SRR5720329',
'sar202.vs.SRR5720322.sorted.95pc.bam':'SRR5720322',
'sar202.vs.SRR5720332.sorted.95pc.bam':'SRR5720332',
'sar202.vs.SRR5720323.sorted.95pc.bam':'SRR5720323',
'sar202.vs.SRR5720326.sorted.95pc.bam':'SRR5720326',
'sar202.vs.SRR5720334.sorted.95pc.bam':'SRR5720334',
'sar202.vs.SRR5720324.sorted.95pc.bam':'SRR5720324',
'sar202.vs.SRR5720337.sorted.95pc.bam':'SRR5720337',
'sar202.vs.SRR5720318.sorted.95pc.bam':'SRR5720318',
'sar202.vs.SRR5720328.sorted.95pc.bam':'SRR5720328',
'sar202.vs.SRR5720335.sorted.95pc.bam':'SRR5720335',
'sar202.vs.SRR5720333.sorted.95pc.bam':'SRR5720333',
'sar202.vs.SRR5720331.sorted.95pc.bam':'SRR5720331',
'sar202.vs.SRR5720342.sorted.95pc.bam':'SRR5720342',
'sar202.vs.SRR5787989.sorted.95pc.bam':'SRR5787989',
'sar202.vs.SRR5787990.sorted.95pc.bam':'SRR5787990',
'sar202.vs.SRR5787991.sorted.95pc.bam':'SRR5787991',
'sar202.vs.SRR5720340.sorted.95pc.bam':'SRR5720340',
'sar202.vs.SRR5720338.sorted.95pc.bam':'SRR5720338',
'sar202.vs.SRR5720336.sorted.95pc.bam':'SRR5720336',
'sar202.vs.SRR5720343.sorted.95pc.bam':'SRR5720343',
'sar202.vs.SRR5720339.sorted.95pc.bam':'SRR5720339',
'sar202.vs.SRR5787992.sorted.95pc.bam':'SRR5787992',
'sar202.vs.SRR5787995.sorted.95pc.bam':'SRR5787995',
'sar202.vs.SRR5787997.sorted.95pc.bam':'SRR5787997',
'sar202.vs.SRR5787996.sorted.95pc.bam':'SRR5787996',
'sar202.vs.SRR5787998.sorted.95pc.bam':'SRR5787998',
'sar202.vs.SRR5788001.sorted.95pc.bam':'SRR5788001',
'sar202.vs.SRR5787993.sorted.95pc.bam':'SRR5787993',
'sar202.vs.SRR5788000.sorted.95pc.bam':'SRR5788000',
'sar202.vs.SRR5720341.sorted.95pc.bam':'SRR5720341',
'sar202.vs.SRR5788005.sorted.95pc.bam':'SRR5788005',
'sar202.vs.SRR5788006.sorted.95pc.bam':'SRR5788006',
'sar202.vs.SRR5787994.sorted.95pc.bam':'SRR5787994',
'sar202.vs.SRR5788003.sorted.95pc.bam':'SRR5788003',
'sar202.vs.SRR5788004.sorted.95pc.bam':'SRR5788004',
'sar202.vs.SRR5787999.sorted.95pc.bam':'SRR5787999',
'sar202.vs.SRR5720344.sorted.95pc.bam':'SRR5720344',
'sar202.vs.SRR5788012.sorted.95pc.bam':'SRR5788012',
'sar202.vs.SRR5788013.sorted.95pc.bam':'SRR5788013',
'sar202.vs.SRR5788015.sorted.95pc.bam':'SRR5788015',
'sar202.vs.SRR5788016.sorted.95pc.bam':'SRR5788016',
'sar202.vs.SRR5788002.sorted.95pc.bam':'SRR5788002',
'sar202.vs.SRR5788018.sorted.95pc.bam':'SRR5788018',
'sar202.vs.SRR5788019.sorted.95pc.bam':'SRR5788019',
'sar202.vs.SRR5788011.sorted.95pc.bam':'SRR5788011',
'sar202.vs.SRR5788021.sorted.95pc.bam':'SRR5788021',
'sar202.vs.SRR5788022.sorted.95pc.bam':'SRR5788022',
'sar202.vs.SRR5788014.sorted.95pc.bam':'SRR5788014',
'sar202.vs.SRR5788024.sorted.95pc.bam':'SRR5788024',
'sar202.vs.SRR5788009.sorted.95pc.bam':'SRR5788009',
'sar202.vs.SRR5788008.sorted.95pc.bam':'SRR5788008',
'sar202.vs.SRR5788007.sorted.95pc.bam':'SRR5788007',
'sar202.vs.SRR5788026.sorted.95pc.bam':'SRR5788026',
'sar202.vs.SRR5788017.sorted.95pc.bam':'SRR5788017',
'sar202.vs.SRR5788028.sorted.95pc.bam':'SRR5788028',
'sar202.vs.SRR5788023.sorted.95pc.bam':'SRR5788023',
'sar202.vs.SRR5788032.sorted.95pc.bam':'SRR5788032',
'sar202.vs.SRR5788033.sorted.95pc.bam':'SRR5788033',
'sar202.vs.SRR5788025.sorted.95pc.bam':'SRR5788025',
'sar202.vs.SRR5788030.sorted.95pc.bam':'SRR5788030',
'sar202.vs.SRR5788010.sorted.95pc.bam':'SRR5788010',
'sar202.vs.SRR5788034.sorted.95pc.bam':'SRR5788034',
'sar202.vs.SRR5788035.sorted.95pc.bam':'SRR5788035',
'sar202.vs.SRR5788031.sorted.95pc.bam':'SRR5788031',
'sar202.vs.SRR5788029.sorted.95pc.bam':'SRR5788029',
'sar202.vs.SRR5788027.sorted.95pc.bam':'SRR5788027',
'sar202.vs.SRR5788037.sorted.95pc.bam':'SRR5788037',
'sar202.vs.SRR5788041.sorted.95pc.bam':'SRR5788041',
'sar202.vs.SRR5788038.sorted.95pc.bam':'SRR5788038',
'sar202.vs.SRR5788042.sorted.95pc.bam':'SRR5788042',
'sar202.vs.SRR5788039.sorted.95pc.bam':'SRR5788039',
'sar202.vs.SRR5788040.sorted.95pc.bam':'SRR5788040',
'sar202.vs.SRR5788047.sorted.95pc.bam':'SRR5788047',
'sar202.vs.SRR5788045.sorted.95pc.bam':'SRR5788045',
'sar202.vs.SRR5788043.sorted.95pc.bam':'SRR5788043',
'sar202.vs.SRR5788046.sorted.95pc.bam':'SRR5788046',
'sar202.vs.SRR5788044.sorted.95pc.bam':'SRR5788044',
'sar202.vs.SRR5788049.sorted.95pc.bam':'SRR5788049',
'sar202.vs.SRR5788050.sorted.95pc.bam':'SRR5788050',
'sar202.vs.SRR5788052.sorted.95pc.bam':'SRR5788052',
'sar202.vs.SRR5788053.sorted.95pc.bam':'SRR5788053',
'sar202.vs.SRR5788048.sorted.95pc.bam':'SRR5788048',
'sar202.vs.SRR5788054.sorted.95pc.bam':'SRR5788054',
'sar202.vs.SRR5788051.sorted.95pc.bam':'SRR5788051',
'sar202.vs.SRR5788060.sorted.95pc.bam':'SRR5788060',
'sar202.vs.SRR5788055.sorted.95pc.bam':'SRR5788055',
'sar202.vs.SRR5788056.sorted.95pc.bam':'SRR5788056',
'sar202.vs.SRR5788058.sorted.95pc.bam':'SRR5788058',
'sar202.vs.SRR5788059.sorted.95pc.bam':'SRR5788059',
'sar202.vs.SRR5788057.sorted.95pc.bam':'SRR5788057',
'sar202.vs.SRR5788036.sorted.95pc.bam':'SRR5788036',
'sar202.vs.SRR5788061.sorted.95pc.bam':'SRR5788061',
'sar202.vs.SRR5788068.sorted.95pc.bam':'SRR5788068',
'sar202.vs.SRR5788020.sorted.95pc.bam':'SRR5788020',
'sar202.vs.SRR5788065.sorted.95pc.bam':'SRR5788065',
'sar202.vs.SRR5788062.sorted.95pc.bam':'SRR5788062',
'sar202.vs.SRR5788072.sorted.95pc.bam':'SRR5788072',
'sar202.vs.SRR5788073.sorted.95pc.bam':'SRR5788073',
'sar202.vs.SRR5788066.sorted.95pc.bam':'SRR5788066',
'sar202.vs.SRR5788071.sorted.95pc.bam':'SRR5788071',
'sar202.vs.SRR5788075.sorted.95pc.bam':'SRR5788075',
'sar202.vs.SRR5788063.sorted.95pc.bam':'SRR5788063',
'sar202.vs.SRR5788076.sorted.95pc.bam':'SRR5788076',
'sar202.vs.SRR5788077.sorted.95pc.bam':'SRR5788077',
'sar202.vs.SRR5788074.sorted.95pc.bam':'SRR5788074',
'sar202.vs.SRR5788079.sorted.95pc.bam':'SRR5788079',
'sar202.vs.SRR5788078.sorted.95pc.bam':'SRR5788078',
'sar202.vs.SRR5788080.sorted.95pc.bam':'SRR5788080',
'sar202.vs.SRR5788067.sorted.95pc.bam':'SRR5788067',
'sar202.vs.SRR5788085.sorted.95pc.bam':'SRR5788085',
'sar202.vs.SRR5788070.sorted.95pc.bam':'SRR5788070',
'sar202.vs.SRR5788069.sorted.95pc.bam':'SRR5788069',
'sar202.vs.SRR5788087.sorted.95pc.bam':'SRR5788087',
'sar202.vs.SRR5788064.sorted.95pc.bam':'SRR5788064',
'sar202.vs.SRR5788089.sorted.95pc.bam':'SRR5788089',
'sar202.vs.SRR5788083.sorted.95pc.bam':'SRR5788083',
'sar202.vs.SRR5788091.sorted.95pc.bam':'SRR5788091',
'sar202.vs.SRR5788081.sorted.95pc.bam':'SRR5788081',
'sar202.vs.SRR5788092.sorted.95pc.bam':'SRR5788092',
'sar202.vs.SRR5788082.sorted.95pc.bam':'SRR5788082',
'sar202.vs.SRR5788093.sorted.95pc.bam':'SRR5788093',
'sar202.vs.SRR5788095.sorted.95pc.bam':'SRR5788095',
'sar202.vs.SRR5788094.sorted.95pc.bam':'SRR5788094',
'sar202.vs.SRR5788088.sorted.95pc.bam':'SRR5788088',
'sar202.vs.SRR5788097.sorted.95pc.bam':'SRR5788097',
'sar202.vs.SRR5788096.sorted.95pc.bam':'SRR5788096',
'sar202.vs.SRR5788086.sorted.95pc.bam':'SRR5788086',
'sar202.vs.SRR5788100.sorted.95pc.bam':'SRR5788100',
'sar202.vs.SRR5788098.sorted.95pc.bam':'SRR5788098',
'sar202.vs.SRR5788105.sorted.95pc.bam':'SRR5788105',
'sar202.vs.SRR5788104.sorted.95pc.bam':'SRR5788104',
'sar202.vs.SRR5788101.sorted.95pc.bam':'SRR5788101',
'sar202.vs.SRR5788084.sorted.95pc.bam':'SRR5788084',
'sar202.vs.SRR5788102.sorted.95pc.bam':'SRR5788102',
'sar202.vs.SRR5788099.sorted.95pc.bam':'SRR5788099',
'sar202.vs.SRR5788109.sorted.95pc.bam':'SRR5788109',
'sar202.vs.SRR5788090.sorted.95pc.bam':'SRR5788090',
'sar202.vs.SRR5788112.sorted.95pc.bam':'SRR5788112',
'sar202.vs.SRR5788114.sorted.95pc.bam':'SRR5788114',
'sar202.vs.SRR5788113.sorted.95pc.bam':'SRR5788113',
'sar202.vs.SRR5788115.sorted.95pc.bam':'SRR5788115',
'sar202.vs.SRR5788116.sorted.95pc.bam':'SRR5788116',
'sar202.vs.SRR5788110.sorted.95pc.bam':'SRR5788110',
'sar202.vs.SRR5788103.sorted.95pc.bam':'SRR5788103',
'sar202.vs.SRR5788117.sorted.95pc.bam':'SRR5788117',
'sar202.vs.SRR5788118.sorted.95pc.bam':'SRR5788118',
'sar202.vs.SRR5788106.sorted.95pc.bam':'SRR5788106',
'sar202.vs.SRR5788119.sorted.95pc.bam':'SRR5788119',
'sar202.vs.SRR5788120.sorted.95pc.bam':'SRR5788120',
'sar202.vs.SRR5788122.sorted.95pc.bam':'SRR5788122',
'sar202.vs.SRR5788121.sorted.95pc.bam':'SRR5788121',
'sar202.vs.SRR5788123.sorted.95pc.bam':'SRR5788123',
'sar202.vs.SRR5788124.sorted.95pc.bam':'SRR5788124',
'sar202.vs.SRR5788126.sorted.95pc.bam':'SRR5788126',
'sar202.vs.SRR5788125.sorted.95pc.bam':'SRR5788125',
'sar202.vs.SRR5788127.sorted.95pc.bam':'SRR5788127',
'sar202.vs.SRR5788130.sorted.95pc.bam':'SRR5788130',
'sar202.vs.SRR5788128.sorted.95pc.bam':'SRR5788128',
'sar202.vs.SRR5788129.sorted.95pc.bam':'SRR5788129',
'sar202.vs.SRR5788133.sorted.95pc.bam':'SRR5788133',
'sar202.vs.SRR5788136.sorted.95pc.bam':'SRR5788136',
'sar202.vs.SRR5788134.sorted.95pc.bam':'SRR5788134',
'sar202.vs.SRR5788131.sorted.95pc.bam':'SRR5788131',
'sar202.vs.SRR5788137.sorted.95pc.bam':'SRR5788137',
'sar202.vs.SRR5788132.sorted.95pc.bam':'SRR5788132',
'sar202.vs.SRR5788135.sorted.95pc.bam':'SRR5788135',
'sar202.vs.SRR5788108.sorted.95pc.bam':'SRR5788108',
'sar202.vs.SRR5788140.sorted.95pc.bam':'SRR5788140',
'sar202.vs.SRR5788144.sorted.95pc.bam':'SRR5788144',
'sar202.vs.SRR5788111.sorted.95pc.bam':'SRR5788111',
'sar202.vs.SRR5788142.sorted.95pc.bam':'SRR5788142',
'sar202.vs.SRR5788107.sorted.95pc.bam':'SRR5788107',
'sar202.vs.SRR5788147.sorted.95pc.bam':'SRR5788147',
'sar202.vs.SRR5788148.sorted.95pc.bam':'SRR5788148',
'sar202.vs.SRR5788149.sorted.95pc.bam':'SRR5788149',
'sar202.vs.SRR5788141.sorted.95pc.bam':'SRR5788141',
'sar202.vs.SRR5788145.sorted.95pc.bam':'SRR5788145',
'sar202.vs.SRR5788152.sorted.95pc.bam':'SRR5788152',
'sar202.vs.SRR5788154.sorted.95pc.bam':'SRR5788154',
'sar202.vs.SRR5788138.sorted.95pc.bam':'SRR5788138',
'sar202.vs.SRR5788155.sorted.95pc.bam':'SRR5788155',
'sar202.vs.SRR5788153.sorted.95pc.bam':'SRR5788153',
'sar202.vs.SRR5788158.sorted.95pc.bam':'SRR5788158',
'sar202.vs.SRR5788156.sorted.95pc.bam':'SRR5788156',
'sar202.vs.SRR5788151.sorted.95pc.bam':'SRR5788151',
'sar202.vs.SRR5788157.sorted.95pc.bam':'SRR5788157',
'sar202.vs.SRR5788139.sorted.95pc.bam':'SRR5788139',
'sar202.vs.SRR5788143.sorted.95pc.bam':'SRR5788143',
'sar202.vs.SRR5788162.sorted.95pc.bam':'SRR5788162',
'sar202.vs.SRR5788150.sorted.95pc.bam':'SRR5788150',
'sar202.vs.SRR5788159.sorted.95pc.bam':'SRR5788159',
'sar202.vs.SRR5788163.sorted.95pc.bam':'SRR5788163',
'sar202.vs.SRR5788160.sorted.95pc.bam':'SRR5788160',
'sar202.vs.SRR5788164.sorted.95pc.bam':'SRR5788164',
'sar202.vs.SRR5788165.sorted.95pc.bam':'SRR5788165',
'sar202.vs.SRR5788166.sorted.95pc.bam':'SRR5788166',
'sar202.vs.SRR5788167.sorted.95pc.bam':'SRR5788167',
'sar202.vs.SRR5788172.sorted.95pc.bam':'SRR5788172',
'sar202.vs.SRR5788161.sorted.95pc.bam':'SRR5788161',
'sar202.vs.SRR5788171.sorted.95pc.bam':'SRR5788171',
'sar202.vs.SRR5788146.sorted.95pc.bam':'SRR5788146',
'sar202.vs.SRR5788170.sorted.95pc.bam':'SRR5788170',
'sar202.vs.SRR5788176.sorted.95pc.bam':'SRR5788176',
'sar202.vs.SRR5788178.sorted.95pc.bam':'SRR5788178',
'sar202.vs.SRR5788177.sorted.95pc.bam':'SRR5788177',
'sar202.vs.SRR5788173.sorted.95pc.bam':'SRR5788173',
'sar202.vs.SRR5788182.sorted.95pc.bam':'SRR5788182',
'sar202.vs.SRR5788181.sorted.95pc.bam':'SRR5788181',
'sar202.vs.SRR5788184.sorted.95pc.bam':'SRR5788184',
'sar202.vs.SRR5788183.sorted.95pc.bam':'SRR5788183',
'sar202.vs.SRR5788179.sorted.95pc.bam':'SRR5788179',
'sar202.vs.SRR5788187.sorted.95pc.bam':'SRR5788187',
'sar202.vs.SRR5788188.sorted.95pc.bam':'SRR5788188',
'sar202.vs.SRR5788174.sorted.95pc.bam':'SRR5788174',
'sar202.vs.SRR5788180.sorted.95pc.bam':'SRR5788180',
'sar202.vs.SRR5788169.sorted.95pc.bam':'SRR5788169',
'sar202.vs.SRR5788191.sorted.95pc.bam':'SRR5788191',
'sar202.vs.SRR5788168.sorted.95pc.bam':'SRR5788168',
'sar202.vs.SRR5788190.sorted.95pc.bam':'SRR5788190',
'sar202.vs.SRR5788194.sorted.95pc.bam':'SRR5788194',
'sar202.vs.SRR5788185.sorted.95pc.bam':'SRR5788185',
'sar202.vs.SRR5788175.sorted.95pc.bam':'SRR5788175',
'sar202.vs.SRR5788189.sorted.95pc.bam':'SRR5788189',
'sar202.vs.SRR5788186.sorted.95pc.bam':'SRR5788186',
'sar202.vs.SRR5788197.sorted.95pc.bam':'SRR5788197',
'sar202.vs.SRR5788196.sorted.95pc.bam':'SRR5788196',
'sar202.vs.SRR5788199.sorted.95pc.bam':'SRR5788199',
'sar202.vs.SRR5788193.sorted.95pc.bam':'SRR5788193',
'sar202.vs.SRR5788195.sorted.95pc.bam':'SRR5788195',
'sar202.vs.SRR5788202.sorted.95pc.bam':'SRR5788202',
'sar202.vs.SRR5788205.sorted.95pc.bam':'SRR5788205',
'sar202.vs.SRR5788203.sorted.95pc.bam':'SRR5788203',
'sar202.vs.SRR5788206.sorted.95pc.bam':'SRR5788206',
'sar202.vs.SRR5788208.sorted.95pc.bam':'SRR5788208',
'sar202.vs.SRR5788207.sorted.95pc.bam':'SRR5788207',
'sar202.vs.SRR5788209.sorted.95pc.bam':'SRR5788209',
'sar202.vs.SRR5788200.sorted.95pc.bam':'SRR5788200',
'sar202.vs.SRR5788210.sorted.95pc.bam':'SRR5788210',
'sar202.vs.SRR5788211.sorted.95pc.bam':'SRR5788211',
'sar202.vs.SRR5788212.sorted.95pc.bam':'SRR5788212',
'sar202.vs.SRR5788192.sorted.95pc.bam':'SRR5788192',
'sar202.vs.SRR5788198.sorted.95pc.bam':'SRR5788198',
'sar202.vs.SRR5788201.sorted.95pc.bam':'SRR5788201',
'sar202.vs.SRR5788213.sorted.95pc.bam':'SRR5788213',
'sar202.vs.SRR5788214.sorted.95pc.bam':'SRR5788214',
'sar202.vs.SRR5788204.sorted.95pc.bam':'SRR5788204',
'sar202.vs.SRR5788215.sorted.95pc.bam':'SRR5788215',
'sar202.vs.SRR5788218.sorted.95pc.bam':'SRR5788218',
'sar202.vs.SRR5788216.sorted.95pc.bam':'SRR5788216',
'sar202.vs.SRR5788217.sorted.95pc.bam':'SRR5788217',
'sar202.vs.SRR5788224.sorted.95pc.bam':'SRR5788224',
'sar202.vs.SRR5788219.sorted.95pc.bam':'SRR5788219',
'sar202.vs.SRR5788221.sorted.95pc.bam':'SRR5788221',
'sar202.vs.SRR5788229.sorted.95pc.bam':'SRR5788229',
'sar202.vs.SRR5788230.sorted.95pc.bam':'SRR5788230',
'sar202.vs.SRR5788222.sorted.95pc.bam':'SRR5788222',
'sar202.vs.SRR5788223.sorted.95pc.bam':'SRR5788223',
'sar202.vs.SRR5788233.sorted.95pc.bam':'SRR5788233',
'sar202.vs.SRR5788234.sorted.95pc.bam':'SRR5788234',
'sar202.vs.SRR5788235.sorted.95pc.bam':'SRR5788235',
'sar202.vs.SRR5788236.sorted.95pc.bam':'SRR5788236',
'sar202.vs.SRR5788237.sorted.95pc.bam':'SRR5788237',
'sar202.vs.SRR5788220.sorted.95pc.bam':'SRR5788220',
'sar202.vs.SRR5788238.sorted.95pc.bam':'SRR5788238',
'sar202.vs.SRR5788227.sorted.95pc.bam':'SRR5788227',
'sar202.vs.SRR5788239.sorted.95pc.bam':'SRR5788239',
'sar202.vs.SRR5788240.sorted.95pc.bam':'SRR5788240',
'sar202.vs.SRR5788232.sorted.95pc.bam':'SRR5788232',
'sar202.vs.SRR5788241.sorted.95pc.bam':'SRR5788241',
'sar202.vs.SRR5788245.sorted.95pc.bam':'SRR5788245',
'sar202.vs.SRR5788231.sorted.95pc.bam':'SRR5788231',
'sar202.vs.SRR5788246.sorted.95pc.bam':'SRR5788246',
'sar202.vs.SRR5788247.sorted.95pc.bam':'SRR5788247',
'sar202.vs.SRR5788244.sorted.95pc.bam':'SRR5788244',
'sar202.vs.SRR5788248.sorted.95pc.bam':'SRR5788248',
'sar202.vs.SRR5788225.sorted.95pc.bam':'SRR5788225',
'sar202.vs.SRR5788252.sorted.95pc.bam':'SRR5788252',
'sar202.vs.SRR5788242.sorted.95pc.bam':'SRR5788242',
'sar202.vs.SRR5788226.sorted.95pc.bam':'SRR5788226',
'sar202.vs.SRR5788253.sorted.95pc.bam':'SRR5788253',
'sar202.vs.SRR5788255.sorted.95pc.bam':'SRR5788255',
'sar202.vs.SRR5788257.sorted.95pc.bam':'SRR5788257',
'sar202.vs.SRR5788256.sorted.95pc.bam':'SRR5788256',
'sar202.vs.SRR5788254.sorted.95pc.bam':'SRR5788254',
'sar202.vs.SRR5788258.sorted.95pc.bam':'SRR5788258',
'sar202.vs.SRR5788259.sorted.95pc.bam':'SRR5788259',
'sar202.vs.SRR5788260.sorted.95pc.bam':'SRR5788260',
'sar202.vs.SRR5788243.sorted.95pc.bam':'SRR5788243',
'sar202.vs.SRR5788261.sorted.95pc.bam':'SRR5788261',
'sar202.vs.SRR5788264.sorted.95pc.bam':'SRR5788264',
'sar202.vs.SRR5788262.sorted.95pc.bam':'SRR5788262',
'sar202.vs.SRR5788263.sorted.95pc.bam':'SRR5788263',
'sar202.vs.SRR5788228.sorted.95pc.bam':'SRR5788228',
'sar202.vs.SRR5788268.sorted.95pc.bam':'SRR5788268',
'sar202.vs.SRR5788249.sorted.95pc.bam':'SRR5788249',
'sar202.vs.SRR5788269.sorted.95pc.bam':'SRR5788269',
'sar202.vs.SRR5788266.sorted.95pc.bam':'SRR5788266',
'sar202.vs.SRR5788270.sorted.95pc.bam':'SRR5788270',
'sar202.vs.SRR5788251.sorted.95pc.bam':'SRR5788251',
'sar202.vs.SRR5788271.sorted.95pc.bam':'SRR5788271',
'sar202.vs.SRR5788273.sorted.95pc.bam':'SRR5788273',
'sar202.vs.SRR5788275.sorted.95pc.bam':'SRR5788275',
'sar202.vs.SRR5788276.sorted.95pc.bam':'SRR5788276',
'sar202.vs.SRR5788277.sorted.95pc.bam':'SRR5788277',
'sar202.vs.SRR5788274.sorted.95pc.bam':'SRR5788274',
'sar202.vs.SRR5788279.sorted.95pc.bam':'SRR5788279',
'sar202.vs.SRR5788250.sorted.95pc.bam':'SRR5788250',
'sar202.vs.SRR5788278.sorted.95pc.bam':'SRR5788278',
'sar202.vs.SRR5788284.sorted.95pc.bam':'SRR5788284',
'sar202.vs.SRR5788280.sorted.95pc.bam':'SRR5788280',
'sar202.vs.SRR5788267.sorted.95pc.bam':'SRR5788267',
'sar202.vs.SRR5788283.sorted.95pc.bam':'SRR5788283',
'sar202.vs.SRR5788287.sorted.95pc.bam':'SRR5788287',
'sar202.vs.SRR5788265.sorted.95pc.bam':'SRR5788265',
'sar202.vs.SRR5788286.sorted.95pc.bam':'SRR5788286',
'sar202.vs.SRR5788285.sorted.95pc.bam':'SRR5788285',
'sar202.vs.SRR5788291.sorted.95pc.bam':'SRR5788291',
'sar202.vs.SRR5788272.sorted.95pc.bam':'SRR5788272',
'sar202.vs.SRR5788290.sorted.95pc.bam':'SRR5788290',
'sar202.vs.SRR5788293.sorted.95pc.bam':'SRR5788293',
'sar202.vs.SRR5788294.sorted.95pc.bam':'SRR5788294',
'sar202.vs.SRR5788296.sorted.95pc.bam':'SRR5788296',
'sar202.vs.SRR5788295.sorted.95pc.bam':'SRR5788295',
'sar202.vs.SRR5788288.sorted.95pc.bam':'SRR5788288',
'sar202.vs.SRR5788292.sorted.95pc.bam':'SRR5788292',
'sar202.vs.SRR5788297.sorted.95pc.bam':'SRR5788297',
'sar202.vs.SRR5788302.sorted.95pc.bam':'SRR5788302',
'sar202.vs.SRR5788298.sorted.95pc.bam':'SRR5788298',
'sar202.vs.SRR5788299.sorted.95pc.bam':'SRR5788299',
'sar202.vs.SRR5788300.sorted.95pc.bam':'SRR5788300',
'sar202.vs.SRR5788303.sorted.95pc.bam':'SRR5788303',
'sar202.vs.SRR5788304.sorted.95pc.bam':'SRR5788304',
'sar202.vs.SRR5788308.sorted.95pc.bam':'SRR5788308',
'sar202.vs.SRR5788309.sorted.95pc.bam':'SRR5788309',
'sar202.vs.SRR5788301.sorted.95pc.bam':'SRR5788301',
'sar202.vs.SRR5788310.sorted.95pc.bam':'SRR5788310',
'sar202.vs.SRR5788289.sorted.95pc.bam':'SRR5788289',
'sar202.vs.SRR5788311.sorted.95pc.bam':'SRR5788311',
'sar202.vs.SRR5788282.sorted.95pc.bam':'SRR5788282',
'sar202.vs.SRR5788307.sorted.95pc.bam':'SRR5788307',
'sar202.vs.SRR5788305.sorted.95pc.bam':'SRR5788305',
'sar202.vs.SRR5788281.sorted.95pc.bam':'SRR5788281',
'sar202.vs.SRR5788306.sorted.95pc.bam':'SRR5788306',
'sar202.vs.SRR5788313.sorted.95pc.bam':'SRR5788313',
'sar202.vs.SRR5788312.sorted.95pc.bam':'SRR5788312',
'sar202.vs.SRR5788316.sorted.95pc.bam':'SRR5788316',
'sar202.vs.SRR5788317.sorted.95pc.bam':'SRR5788317',
'sar202.vs.SRR5788318.sorted.95pc.bam':'SRR5788318',
'sar202.vs.SRR5788324.sorted.95pc.bam':'SRR5788324',
'sar202.vs.SRR5788315.sorted.95pc.bam':'SRR5788315',
'sar202.vs.SRR5788325.sorted.95pc.bam':'SRR5788325',
'sar202.vs.SRR5788326.sorted.95pc.bam':'SRR5788326',
'sar202.vs.SRR5788320.sorted.95pc.bam':'SRR5788320',
'sar202.vs.SRR5788321.sorted.95pc.bam':'SRR5788321',
'sar202.vs.SRR5788314.sorted.95pc.bam':'SRR5788314',
'sar202.vs.SRR5788331.sorted.95pc.bam':'SRR5788331',
'sar202.vs.SRR5788330.sorted.95pc.bam':'SRR5788330',
'sar202.vs.SRR5788319.sorted.95pc.bam':'SRR5788319',
'sar202.vs.SRR5788323.sorted.95pc.bam':'SRR5788323',
'sar202.vs.SRR5788333.sorted.95pc.bam':'SRR5788333',
'sar202.vs.SRR5788335.sorted.95pc.bam':'SRR5788335',
'sar202.vs.SRR5788334.sorted.95pc.bam':'SRR5788334',
'sar202.vs.SRR5788332.sorted.95pc.bam':'SRR5788332',
'sar202.vs.SRR5788327.sorted.95pc.bam':'SRR5788327',
'sar202.vs.SRR5788339.sorted.95pc.bam':'SRR5788339',
'sar202.vs.SRR5788322.sorted.95pc.bam':'SRR5788322',
'sar202.vs.SRR5788329.sorted.95pc.bam':'SRR5788329',
'sar202.vs.SRR5788340.sorted.95pc.bam':'SRR5788340',
'sar202.vs.SRR5788343.sorted.95pc.bam':'SRR5788343',
'sar202.vs.SRR5788345.sorted.95pc.bam':'SRR5788345',
'sar202.vs.SRR5788346.sorted.95pc.bam':'SRR5788346',
'sar202.vs.SRR5788347.sorted.95pc.bam':'SRR5788347',
'sar202.vs.SRR5788342.sorted.95pc.bam':'SRR5788342',
'sar202.vs.SRR5788349.sorted.95pc.bam':'SRR5788349',
'sar202.vs.SRR5788350.sorted.95pc.bam':'SRR5788350',
'sar202.vs.SRR5788341.sorted.95pc.bam':'SRR5788341',
'sar202.vs.SRR5788344.sorted.95pc.bam':'SRR5788344',
'sar202.vs.SRR5788338.sorted.95pc.bam':'SRR5788338',
'sar202.vs.SRR5788336.sorted.95pc.bam':'SRR5788336',
'sar202.vs.SRR5788354.sorted.95pc.bam':'SRR5788354',
'sar202.vs.SRR5788337.sorted.95pc.bam':'SRR5788337',
'sar202.vs.SRR5788352.sorted.95pc.bam':'SRR5788352',
'sar202.vs.SRR5788356.sorted.95pc.bam':'SRR5788356',
'sar202.vs.SRR5788355.sorted.95pc.bam':'SRR5788355',
'sar202.vs.SRR5788358.sorted.95pc.bam':'SRR5788358',
'sar202.vs.SRR5788328.sorted.95pc.bam':'SRR5788328',
'sar202.vs.SRR5788353.sorted.95pc.bam':'SRR5788353',
'sar202.vs.SRR5788351.sorted.95pc.bam':'SRR5788351',
'sar202.vs.SRR5788359.sorted.95pc.bam':'SRR5788359',
'sar202.vs.SRR5788363.sorted.95pc.bam':'SRR5788363',
'sar202.vs.SRR5788361.sorted.95pc.bam':'SRR5788361',
'sar202.vs.SRR5788367.sorted.95pc.bam':'SRR5788367',
'sar202.vs.SRR5788368.sorted.95pc.bam':'SRR5788368',
'sar202.vs.SRR5788362.sorted.95pc.bam':'SRR5788362',
'sar202.vs.SRR5788348.sorted.95pc.bam':'SRR5788348',
'sar202.vs.SRR5788364.sorted.95pc.bam':'SRR5788364',
'sar202.vs.SRR5788365.sorted.95pc.bam':'SRR5788365',
'sar202.vs.SRR5788371.sorted.95pc.bam':'SRR5788371',
'sar202.vs.SRR5788360.sorted.95pc.bam':'SRR5788360',
'sar202.vs.SRR5788375.sorted.95pc.bam':'SRR5788375',
'sar202.vs.SRR5788372.sorted.95pc.bam':'SRR5788372',
'sar202.vs.SRR5788357.sorted.95pc.bam':'SRR5788357',
'sar202.vs.SRR5788376.sorted.95pc.bam':'SRR5788376',
'sar202.vs.SRR5788377.sorted.95pc.bam':'SRR5788377',
'sar202.vs.SRR5788366.sorted.95pc.bam':'SRR5788366',
'sar202.vs.SRR5788380.sorted.95pc.bam':'SRR5788380',
'sar202.vs.SRR5788373.sorted.95pc.bam':'SRR5788373',
'sar202.vs.SRR5788382.sorted.95pc.bam':'SRR5788382',
'sar202.vs.SRR5788384.sorted.95pc.bam':'SRR5788384',
'sar202.vs.SRR5788369.sorted.95pc.bam':'SRR5788369',
'sar202.vs.SRR5788385.sorted.95pc.bam':'SRR5788385',
'sar202.vs.SRR5788374.sorted.95pc.bam':'SRR5788374',
'sar202.vs.SRR5788387.sorted.95pc.bam':'SRR5788387',
'sar202.vs.SRR5788386.sorted.95pc.bam':'SRR5788386',
'sar202.vs.SRR5788370.sorted.95pc.bam':'SRR5788370',
'sar202.vs.SRR5788389.sorted.95pc.bam':'SRR5788389',
'sar202.vs.SRR5788390.sorted.95pc.bam':'SRR5788390',
'sar202.vs.SRR5788392.sorted.95pc.bam':'SRR5788392',
'sar202.vs.SRR5788393.sorted.95pc.bam':'SRR5788393',
'sar202.vs.SRR5788391.sorted.95pc.bam':'SRR5788391',
'sar202.vs.SRR5788394.sorted.95pc.bam':'SRR5788394',
'sar202.vs.SRR5788396.sorted.95pc.bam':'SRR5788396',
'sar202.vs.SRR5788388.sorted.95pc.bam':'SRR5788388',
'sar202.vs.SRR5788378.sorted.95pc.bam':'SRR5788378',
'sar202.vs.SRR5788397.sorted.95pc.bam':'SRR5788397',
'sar202.vs.SRR5788398.sorted.95pc.bam':'SRR5788398',
'sar202.vs.SRR5788381.sorted.95pc.bam':'SRR5788381',
'sar202.vs.SRR5788399.sorted.95pc.bam':'SRR5788399',
'sar202.vs.SRR5788401.sorted.95pc.bam':'SRR5788401',
'sar202.vs.SRR5788400.sorted.95pc.bam':'SRR5788400',
'sar202.vs.SRR5788383.sorted.95pc.bam':'SRR5788383',
'sar202.vs.SRR5788405.sorted.95pc.bam':'SRR5788405',
'sar202.vs.SRR5788406.sorted.95pc.bam':'SRR5788406',
'sar202.vs.SRR5788402.sorted.95pc.bam':'SRR5788402',
'sar202.vs.SRR5788379.sorted.95pc.bam':'SRR5788379',
'sar202.vs.SRR5788395.sorted.95pc.bam':'SRR5788395',
'sar202.vs.SRR5788411.sorted.95pc.bam':'SRR5788411',
'sar202.vs.SRR5788403.sorted.95pc.bam':'SRR5788403',
'sar202.vs.SRR5788407.sorted.95pc.bam':'SRR5788407',
'sar202.vs.SRR5788404.sorted.95pc.bam':'SRR5788404',
'sar202.vs.SRR5788415.sorted.95pc.bam':'SRR5788415',
'sar202.vs.SRR5788416.sorted.95pc.bam':'SRR5788416',
'sar202.vs.SRR5788417.sorted.95pc.bam':'SRR5788417',
'sar202.vs.SRR5788409.sorted.95pc.bam':'SRR5788409',
'sar202.vs.SRR5788418.sorted.95pc.bam':'SRR5788418',
'sar202.vs.SRR5788419.sorted.95pc.bam':'SRR5788419',
'sar202.vs.SRR5788408.sorted.95pc.bam':'SRR5788408',
'sar202.vs.SRR5788410.sorted.95pc.bam':'SRR5788410',
'sar202.vs.SRR5788423.sorted.95pc.bam':'SRR5788423',
'sar202.vs.SRR5788413.sorted.95pc.bam':'SRR5788413',
'sar202.vs.SRR5788412.sorted.95pc.bam':'SRR5788412',
'sar202.vs.SRR5788424.sorted.95pc.bam':'SRR5788424',
'sar202.vs.SRR5788427.sorted.95pc.bam':'SRR5788427',
'sar202.vs.SRR5788428.sorted.95pc.bam':'SRR5788428',
'sar202.vs.SRR5788420.sorted.95pc.bam':'SRR5788420',
'sar202.vs.SRR5788426.sorted.95pc.bam':'SRR5788426',
'sar202.vs.SRR5788425.sorted.95pc.bam':'SRR5788425',
'sar202.vs.SRR5788422.sorted.95pc.bam':'SRR5788422',
'sar202.vs.SRR5788414.sorted.95pc.bam':'SRR5788414',
'sar202.vs.SRR5788434.sorted.95pc.bam':'SRR5788434',
'sar202.vs.SRR5788429.sorted.95pc.bam':'SRR5788429',
'sar202.vs.SRR5788433.sorted.95pc.bam':'SRR5788433',
'sar202.vs.SRR5788437.sorted.95pc.bam':'SRR5788437',
'sar202.vs.SRR5788438.sorted.95pc.bam':'SRR5788438',
'sar202.vs.SRR5788431.sorted.95pc.bam':'SRR5788431',
'sar202.vs.SRR5788439.sorted.95pc.bam':'SRR5788439',
'sar202.vs.SRR5788440.sorted.95pc.bam':'SRR5788440',
'sar202.vs.SRR5788442.sorted.95pc.bam':'SRR5788442',
'sar202.vs.SRR5788441.sorted.95pc.bam':'SRR5788441',
'sar202.vs.SRR5788445.sorted.95pc.bam':'SRR5788445',
'sar202.vs.SRR5788443.sorted.95pc.bam':'SRR5788443',
'sar202.vs.SRR5788430.sorted.95pc.bam':'SRR5788430',
'sar202.vs.SRR5788447.sorted.95pc.bam':'SRR5788447',
'sar202.vs.SRR5788446.sorted.95pc.bam':'SRR5788446',
'sar202.vs.SRR5788444.sorted.95pc.bam':'SRR5788444',
'sar202.vs.SRR5788432.sorted.95pc.bam':'SRR5788432',
'sar202.vs.SRR5788451.sorted.95pc.bam':'SRR5788451',
'sar202.vs.SRR5788450.sorted.95pc.bam':'SRR5788450',
'sar202.vs.SRR5788421.sorted.95pc.bam':'SRR5788421',
'sar202.vs.SRR5788436.sorted.95pc.bam':'SRR5788436',
'sar202.vs.SRR5788449.sorted.95pc.bam':'SRR5788449',
'sar202.vs.SRR5788435.sorted.95pc.bam':'SRR5788435',
'sar202.vs.SRR5788457.sorted.95pc.bam':'SRR5788457',
'sar202.vs.SRR5788458.sorted.95pc.bam':'SRR5788458',
'sar202.vs.SRR5788459.sorted.95pc.bam':'SRR5788459',
'sar202.vs.SRR5788460.sorted.95pc.bam':'SRR5788460',
'sar202.vs.SRR5788448.sorted.95pc.bam':'SRR5788448',
'sar202.vs.SRR5788454.sorted.95pc.bam':'SRR5788454',
'sar202.vs.SRR5788461.sorted.95pc.bam':'SRR5788461',
'sar202.vs.SRR5788463.sorted.95pc.bam':'SRR5788463',
'sar202.vs.SRR5788453.sorted.95pc.bam':'SRR5788453',
'sar202.vs.SRR5788466.sorted.95pc.bam':'SRR5788466',
'sar202.vs.SRR5788462.sorted.95pc.bam':'SRR5788462',
'sar202.vs.SRR5788465.sorted.95pc.bam':'SRR5788465',
'sar202.vs.SRR5788455.sorted.95pc.bam':'SRR5788455',
'sar202.vs.SRR5788468.sorted.95pc.bam':'SRR5788468',
'sar202.vs.SRR6507279.sorted.95pc.bam':'SRR6507279',
'sar202.vs.SRR5788464.sorted.95pc.bam':'SRR5788464',
'sar202.vs.SRR6507277.sorted.95pc.bam':'SRR6507277',
'sar202.vs.SRR5788452.sorted.95pc.bam':'SRR5788452',
'sar202.vs.SRR5788456.sorted.95pc.bam':'SRR5788456',
'sar202.vs.SRR6507278.sorted.95pc.bam':'SRR6507278',
'sar202.vs.SRR6507280.sorted.95pc.bam':'SRR6507280',
'sar202.vs.SRR5788467.sorted.95pc.bam':'SRR5788467',}, inplace=True)
df

,#contig,Length,SRR5720222,SRR5720219,SRR5720227,SRR5720228,SRR5720220,SRR5720223,SRR5720221,SRR5720230,...,SRR5788455,SRR5788468,SRR6507279,SRR5788464,SRR6507277,SRR5788452,SRR5788456,SRR6507278,SRR6507280,SRR5788467
0,LSU_GOM1_000000000001,17893,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,...,0.0000,0.0,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
1,LSU_GOM1_000000000002,3164,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,...,0.0000,0.0,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2,LSU_GOM1_000000000003,14152,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,...,0.0000,0.0,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
3,LSU_GOM1_000000000004,5298,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,...,0.0000,0.0,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
4,LSU_GOM1_000000000005,18323,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,...,0.0000,0.0,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
5,LSU_GOM1_000000000006,8039,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,...,0.0000,0.0,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
6,LSU_GOM1_000000000007,7574,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,...,0.0000,0.0,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
7,LSU_GOM1_000000000008,5392,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,...,0.0000,0.0,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
8,LSU_GOM1_000000000009,19342,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,...,0.0000,0.0,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
9,LSU_GOM1_000000000010,4755,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,...,0.0000,0.0,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000


In [49]:
df.set_index('#contig', inplace=True)

In [50]:
df_transpose=df.T

In [ ]:
#df_transpose.to_csv("/home/minion/Desktop/holger/sar202_coverage_transposed.tsv", sep = '\t')

In [17]:
# in bash
for i in `ls`; \
do echo -n $i >> read_counts_all.tsv; \
echo -ne '\t' >> read_counts_all.tsv; \
cat $i >> read_counts_all.tsv; \
done

SyntaxError: invalid syntax (<ipython-input-17-7324f1b4bdc1>, line 2)

In [51]:
rc = pd.read_csv("/home/minion/Desktop/holger/read_counts/read_counts_all.tsv", sep = '\t')
rc.set_index('#contig', inplace=True)
rc
yay = df_transpose.join(rc)
yay

,LSU_GOM1_000000000001,LSU_GOM1_000000000002,LSU_GOM1_000000000003,LSU_GOM1_000000000004,LSU_GOM1_000000000005,LSU_GOM1_000000000006,LSU_GOM1_000000000007,LSU_GOM1_000000000008,LSU_GOM1_000000000009,LSU_GOM1_000000000010,...,USC_TB94_000000000016,USC_TB94_000000000017,USC_TB94_000000000018,USC_TB94_000000000019,USC_TB94_000000000020,USC_TB94_000000000021,USC_TB94_000000000022,USC_TB94_000000000024,USC_TB94_000000000025,read_count
Length,17893.0,3164.0,14152.0,5298.0,18323.0,8039.0,7574.0,5392.0,19342.0,4755.0,...,33094.0000,33244.0000,81063.0000,26675.0000,35166.0000,54835.0000,24537.0000,34975.0000,23382.0,NaN
SRR5720222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,26379124.0
SRR5720219,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,22377770.0
SRR5720227,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,22370133.0
SRR5720228,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,31512521.0
SRR5720220,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,18532257.0
SRR5720223,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,29188845.0
SRR5720221,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0008,0.0008,0.0006,0.0003,0.0009,0.0006,0.0006,0.0003,0.0,25125604.0
SRR5720230,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,33880679.0
SRR5720225,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,25291252.0


In [52]:
yay_transpose = yay.T

In [53]:
yay_transpose.to_csv("/home/minion/Desktop/holger/sar202_with_read_counts.tsv", sep = '\t')

In [ ]:
#bash
for i in `ls ~/projects/heatmaps/coverage/`; \
do filename=$(basename $i .coverage.tsv); \
cat read_count_final.tsv | grep "$filename" > coverages/"$filename"_coverage.tsv; \
done

In [ ]:
for i in $(cat /home/minion/Desktop/holger/viral_contig_names.txt); \
do cat read_count_final.csv | egrep "contig|$i" > coverages/viral_coverages.tsv; \
done

In [ ]:
file_list = glob.glob('./maxbin/*.fasta') #file_list defined as anything in maxbin filepath with fasta in it using glob package for pathing
for f in file_list: #for every item in the file list do
    bin_name = os.path.basename(f).split('.fasta')[0] #make the bin_name the filename minus the word fasta
    for record in SeqIO.parse(f, 'fasta'): #using SeqIO package look for the item name then do
        results.append((record.id, bin_name))#add the results of the parse and only take the record.id (contig name) and the bin_name from above

In [84]:
import pandas as pd
import os
import glob

file_list = glob.glob("/home/minion/Desktop/holger/coverages/*.tsv")
for f in file_list:
    name = os.path.basename(f).split('_coverage.tsv')[0]
    df=pd.read_csv(f, sep = "\t", header = None)
    df2=df.sum()
    df2[0:1] = os.path.basename(f).split('_coverage.tsv')[0]
    df2=pd.DataFrame(df2)
    df2_transposed = df2.T
    df2_transposed.to_csv("/home/minion/Desktop/holger/transformed/"+name+".tsv", sep ='\t', index = False)

In [80]:
for i in `ls`; \
do filename=$(basename $i .tsv); \
cat $i | grep "$filename" >> all_sar202_coverage.tsv; \
done

,0
0,OSU_TB45


In [25]:
import pandas as pd
cellular = pd.read_csv("/home/minion/projects/sar202/holger/sar202_all_cmeans_coverage.tsv", sep = "\t")
viral = pd.read_csv("/home/minion/projects/sar202/holger/sar202_viral_cmeans_coverage.tsv", sep = "\t")


viral.iloc[:,2:]

,sar202-viral.vs.SRR5720219.sorted.95pc.bam,sar202-viral.vs.SRR5720227.sorted.95pc.bam,sar202-viral.vs.SRR5720222.sorted.95pc.bam
0,0.0000,0.0000,0.0000
1,0.0000,0.0000,0.0000
2,0.0000,0.0000,0.0000
3,0.0000,0.0000,0.0000
4,0.0000,0.0000,0.0000
5,0.0000,0.0000,0.0000
6,0.0000,0.0000,0.0000
7,0.0000,0.0000,0.0000
8,0.0000,0.0000,0.0000
9,0.0000,0.0000,0.0000
